In [ ]:
import os
import sys
sys.path.insert(1, os.path.realpath(os.path.pardir))


from utils.augmentations import get_default_transform
from utils import creating_dataset


# Dataset loading & inspection
This notebook shows how to load the datasets used in this challenge and provides some basic statistics about them.

Note that the data includes EMG signal from 8 electrodes in the EMG armband, the predicted variables are the angle of 20 joints in the hand. Inputs are sampled at 200Hz, but the  outputs are intended to be at a subsampled rate of 25Hz (more on that in other notebooks).

Data was acquired in "healthy" and "amputant" subjects (i.e. with limb loss) using the EMG armband in either the left or right arm. 
Your model's predictions will be evaluated on data from one of the two amputant subjects (fedya), but you can use any and all provided data for training your model. For simplicity, the data has been divided in `training` and `test` subsets (not all subjects have `test` data) to evaluate your model's performance during training. The final submission data is held separate and only used when preparing the submission.csv file (see 04_submit_predictions.ipynb).

### Load data
Start by defining a variable to keep track of where the data is saved on your computer and a set of parameters for selecting which data to load. 

In [ ]:
DATA_PATH = r"F:\Dropbox (Personal)\BCII\BCI Challenges\2024 ALVI EMG Decoding\dataset_v2_blocks\dataset_v2_blocks"

You can decide which data to load (e.g. from both `health` and `amputant` subjects). 
The `test_dataset_list` specifies which subset of the data should be used as test set, and it shouldn't change. You can, however, play around with training your model on different subsets of the available training data. 

You can also define a set of `transform` functions to apply to the data before feeding it to the model, or load the default ones.

In [ ]:
data_paths = dict(
    datasets=[DATA_PATH],
    hand_type = ['left', 'right'], # [left, 'right']
    human_type = ['health', 'amputant'], # [amputant, 'health']
    test_dataset_list = ['fedya_tropin_standart_elbow_left']  # don't change this !
)

# define a config object to keep track of data variables
data_config = creating_dataset.DataConfig(**data_paths)

# get transforms
p_transform = 0.1  # probability of applying the transform
transform = get_default_transform(p_transform)

# load the data
train_dataset, test_dataset = creating_dataset.get_datasets(data_config, transform=transform)


### Inspect the data

`train_dataset` and `test_dataset` are instances of the `torch.utils.data.ConcatDataset` class. 


The following code shows the number of batches in each set as well as the size of input and outputs. 
Note that inputs are of shape `n_channels x batch_size` while the outputs are of shape `n_angles x downsampled_batchs_size` where `downsampled_batch_size = batch_size / 200 * 25` to account for downsampling of predictions. 

In [ ]:
print(f"Train dataset size: {len(train_dataset)}, Test dataset size: {len(test_dataset)}")

X, Y = train_dataset[0]
print(f"X shape: {X.shape}, Y shape: {Y.shape}")

You can also generate a video of the hand movements.

In [ ]:
from utils.hand_visualize import Hand, save_animation
from utils.quats_and_angles import get_quats
import numpy as np


batches = [train_dataset[i] for i in range(10)]
Y = np.concatenate([b[1] for b in batches], axis=1)
quats = get_quats(Y)

hand_gt = Hand(quats)
ani = hand_gt.visualize_all_frames()
save_animation(ani, 'test_vis.gif', fps=25,)   # this will save a .gif file